In [1]:
import allel 
import pandas as pd 
import numpy as np

### Parsing the incredibly poorly written LDNe output

The output is a mixture of different lengths whitespaces and symbols, and so requires some parsing....

In [16]:
phase3ldne_list = pd.read_csv("../data/Phase3.LDNe.list")
collections = phase3ldne_list['pop']

In [17]:
phase3ldne_list.head()

,pop
0,AG1000G-AO.Luanda.2009.coluzzii
1,AG1000G-BF-A.Bana.2012.coluzzii
2,AG1000G-BF-A.Bana.2012.gambiae
3,AG1000G-BF-A.Pala.2012.gambiae
4,AG1000G-BF-A.Souroukoudinga.2012.coluzzii


In [18]:
chroms = ['3L']

Ne_Ag = dict()
Ne_chrom = dict()

for name in collections:
    for chrom in chroms:
        df = pd.read_csv(f"LDNe/Ag_LDNe_{name}.{chrom}.out", header=None, sep='\n')
        df = df[0].str.split('\s\=\s', expand=True) #split first column using equal sign
        df.columns = ['one', 'two']
        start = np.where(df.one.str.contains('Harmonic Mean'))[0][0] #find start of results
        end = len(df)-3
        df = df.iloc[start:end]                                           #subset 
        
        #loop through rows and remove multiple whitespaces to one whitespace only, then split to new columns
        res=pd.DataFrame()
        for i in range(5):
            results = ' '.join(df.iloc[i,1].split()).split()
            res = res.append(pd.Series(results),ignore_index=True)
        
        #get columns which have estimates for CIs (poorly parsed)
        df2 = df.iloc[6:9,]
        jack = df.iloc[9,0]
        df2.one = df2.one.str.replace("*", "")
        df2 = df2.one.str.split(expand=True).reset_index(drop=True)
        
        #loop through rows and remove multiple whitespaces to one whitespace only, then split to new columns
        for i in range(len(df2)):
            df2.iloc[i,0] = ' '.join(df2.iloc[i,0].split())
        
        #extract specific estimates,works only because all files are parsed identically, not ideal
        para_1 = df2.loc[0,1:4]
        para_2 = df2.loc[1,0:3]
        jack_1 = df2.loc[2,3:7]
        jack_2 = pd.Series(jack.split())
        cols = df.iloc[:5]['one']
        cols = cols.append(pd.Series(['Parametric CI - lower', 
                              'Parametric CI - upper', 
                              'Jackknife CI - lower', 
                              'Jackknife CI - upper']), ignore_index=True)
        
        #append the  estimates to the results 
        df = res.append(para_1.reset_index(drop=True)).append(para_2.reset_index(drop=True)).append(jack_1.reset_index(drop=True)).append(jack_2, ignore_index=True)
        
        #join results to the parameter column and change column names 
        final_results = pd.concat([cols,df], axis=1)
        final_results.columns = ['Parameter', 
                         'minAF_0.05', 
                         'minAF_0.02', 
                         'minAF_0.01', 
                         'minAF_0+']
        
        #store results in dict and then save within nested dict
        Ne_chrom[chrom] = final_results.iloc[:,:2] ### .iloc[:,:2] for 1 allele frequency only!
        
    Ne_Ag[name] = dict(Ne_chrom)

/home/sanj/apps/anaconda3/envs/NeAg/lib/python3.8/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [20]:
Ne = pd.DataFrame()

# transpose, add population and chromosome columns, then re-order columns, and combine all tables for each
# pop into one big one
for name in collections:
    for chrom in chroms:
        
        Ne_Ag[name][chrom] = Ne_Ag[name][chrom].set_index('Parameter').T
        
        Ne_Ag[name][chrom]['name'] = name
        Ne_Ag[name][chrom]['chrom'] = chrom

                # get a list of columns
        cols = list(Ne_Ag[name][chrom])
        # move the column to head of list using index, pop and insert
        cols.insert(0, cols.pop(cols.index('name')))
        Ne_Ag[name][chrom] = Ne_Ag[name][chrom].loc[:, cols]
        cols = list(Ne_Ag[name][chrom])
        cols.insert(0, cols.pop(cols.index('chrom')))
        Ne_Ag[name][chrom] = Ne_Ag[name][chrom].loc[:, cols]
        Ne_Ag[name][chrom]
        
        Ne = pd.concat([Ne, Ne_Ag[name][chrom]])

#change names removing whitespace
Ne.columns = ['chrom', 'pop', 'sample_size', 'independent_comparisons', 
                      'overall_r^2', 'expected_r^2', 'Ne_estimate', 'Parametric_CI_lower',
                     'Parametric_CI_upper', 'Jackknife_CI_lower', 'Jackknife_CI_upper']

In [23]:
Ne

,chrom,pop,sample_size,independent_comparisons,overall_r^2,expected_r^2,Ne_estimate,Parametric_CI_lower,Parametric_CI_upper,Jackknife_CI_lower,Jackknife_CI_upper
minAF_0.05,3L,AG1000G-AO.Luanda.2009.coluzzii,81.0,187529754,0.014558,0.012832,191.0,190.7,191.3,110.5,526.1
minAF_0.05,3L,AG1000G-BF-A.Bana.2012.coluzzii,42.0,222718988,0.025603,0.025618,Infinite,Infinite,Infinite,Infinite,Infinite
minAF_0.05,3L,AG1000G-BF-A.Bana.2012.gambiae,22.0,208946299,0.052310,0.052201,2830.4,2591.2,3118.2,1829.0,6238.2
minAF_0.05,3L,AG1000G-BF-A.Pala.2012.gambiae,48.0,232037152,0.022405,0.022218,1777.0,1739.1,1816.5,782.8,Infinite
minAF_0.05,3L,AG1000G-BF-A.Souroukoudinga.2012.coluzzii,29.0,221349547,0.038297,0.038355,Infinite,Infinite,Infinite,Infinite,Infinite
minAF_0.05,3L,AG1000G-BF-A.Souroukoudinga.2012.gambiae,28.0,220277734,0.039902,0.039856,6739.5,5793.8,8052.7,3631.1,46290.7
minAF_0.05,3L,AG1000G-BF-B.Bana.2014.coluzzii,47.0,229480914,0.022701,0.022721,Infinite,Infinite,Infinite,Infinite,Infinite
minAF_0.05,3L,AG1000G-BF-B.Pala.2014.gambiae,16.0,208742208,0.078917,0.075831,98.1,97.6,98.6,45.4,Infinite
minAF_0.05,3L,AG1000G-CD.Gbadolite.2015.gambiae,76.0,243285719,0.017050,0.013710,97.7,97.6,97.8,55.9,243.1
minAF_0.05,3L,AG1000G-CF.Bangui.1994.gambiae,53.0,232575954,0.020087,0.020004,3978.4,3812.1,4159.8,2122.8,30273.2


### Write to txt file

In [22]:
Ne.to_csv("Ne_analyses.LDNe.tsv", sep="\t", header=True, index=True, index_label='AF')

### Loading metadata

Now, lets add in all the metadata for each population, so that we can plot this on a map.

In [31]:
manifest = pd.read_csv("../../data/phase3/Ag1000g.phase3.manifest.full.tsv", sep="\t")
manifest.location = [loc.replace(" ", "") for loc in manifest.location]
manifest['pop'] = manifest[['sample_set', 'location','year', 'species_gambiae_coluzzii']].astype(str).apply('.'.join, axis=1)

We need to summarise the full manifest - essentially we want metadata info for each distinct collection we are running LDNe on, but also want AIMs and lat long info, for plotting. We first aggregate `manifest` and get the means of AIMS and long/lats, and then separately run size() on `manifest` and assign that to our new df, `manifest_summary`.

With this df, we can join the LDNe results and that will contain all we need to plot.

In [32]:
manifest_summary = manifest.groupby(['pop', 'sample_set', 'country', 'year', 'location', 'species_gambiae_coluzzii']).agg('mean').drop(columns='month').reset_index()
manifest_summary['counts'] = manifest.groupby(['pop', 'sample_set', 'year', 'location', 'species_gambiae_coluzzii']).size().reset_index(name='counts').counts

Lets write the list of pops with n > 15 to .tsv, `Phase3.LDNe.list`, so we have a list of the pops.

In [206]:
manifest_summary[manifest_summary.counts > 15]['pop'].to_csv("../data/Phase3.LDNe.list", index=None)

In [36]:
manifest_summary[manifest_summary.counts > 15]

,pop,sample_set,country,year,location,species_gambiae_coluzzii,latitude,longitude,aim_fraction_colu,aim_fraction_arab,counts
0,AG1000G-AO.Luanda.2009.coluzzii,AG1000G-AO,Angola,2009,Luanda,coluzzii,-8.884,13.302,0.937901,0.001136,81
1,AG1000G-BF-A.Bana.2012.coluzzii,AG1000G-BF-A,Burkina Faso,2012,Bana,coluzzii,11.233,-4.472,0.975738,0.002024,42
2,AG1000G-BF-A.Bana.2012.gambiae,AG1000G-BF-A,Burkina Faso,2012,Bana,gambiae,11.233,-4.472,0.035000,0.002455,22
5,AG1000G-BF-A.Pala.2012.gambiae,AG1000G-BF-A,Burkina Faso,2012,Pala,gambiae,11.150,-4.235,0.027146,0.002146,48
6,AG1000G-BF-A.Souroukoudinga.2012.coluzzii,AG1000G-BF-A,Burkina Faso,2012,Souroukoudinga,coluzzii,11.235,-4.535,0.977276,0.002793,29
7,AG1000G-BF-A.Souroukoudinga.2012.gambiae,AG1000G-BF-A,Burkina Faso,2012,Souroukoudinga,gambiae,11.235,-4.535,0.028071,0.002071,28
9,AG1000G-BF-B.Bana.2014.coluzzii,AG1000G-BF-B,Burkina Faso,2014,Bana,coluzzii,11.233,-4.472,0.977532,0.002149,47
12,AG1000G-BF-B.Pala.2014.gambiae,AG1000G-BF-B,Burkina Faso,2014,Pala,gambiae,11.150,-4.235,0.027188,0.002188,16
16,AG1000G-CD.Gbadolite.2015.gambiae,AG1000G-CD,Democratic Republic of Congo,2015,Gbadolite,gambiae,4.283,21.017,0.024776,0.002355,76
20,AG1000G-CF.Bangui.1994.gambiae,AG1000G-CF,Central African Republic,1994,Bangui,gambiae,4.367,18.583,0.025943,0.002113,53


In [37]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(manifest_summary[manifest_summary.counts > 8])

,pop,sample_set,country,year,location,species_gambiae_coluzzii,latitude,longitude,aim_fraction_colu,aim_fraction_arab,counts
0,AG1000G-AO.Luanda.2009.coluzzii,AG1000G-AO,Angola,2009,Luanda,coluzzii,-8.884,13.302,0.937901,0.001136,81
1,AG1000G-BF-A.Bana.2012.coluzzii,AG1000G-BF-A,Burkina Faso,2012,Bana,coluzzii,11.233,-4.472,0.975738,0.002024,42
2,AG1000G-BF-A.Bana.2012.gambiae,AG1000G-BF-A,Burkina Faso,2012,Bana,gambiae,11.233,-4.472,0.035000,0.002455,22
4,AG1000G-BF-A.Pala.2012.coluzzii,AG1000G-BF-A,Burkina Faso,2012,Pala,coluzzii,11.150,-4.235,0.979909,0.002000,11
5,AG1000G-BF-A.Pala.2012.gambiae,AG1000G-BF-A,Burkina Faso,2012,Pala,gambiae,11.150,-4.235,0.027146,0.002146,48
6,AG1000G-BF-A.Souroukoudinga.2012.coluzzii,AG1000G-BF-A,Burkina Faso,2012,Souroukoudinga,coluzzii,11.235,-4.535,0.977276,0.002793,29
7,AG1000G-BF-A.Souroukoudinga.2012.gambiae,AG1000G-BF-A,Burkina Faso,2012,Souroukoudinga,gambiae,11.235,-4.535,0.028071,0.002071,28
9,AG1000G-BF-B.Bana.2014.coluzzii,AG1000G-BF-B,Burkina Faso,2014,Bana,coluzzii,11.233,-4.472,0.977532,0.002149,47
10,AG1000G-BF-B.Bana.2014.gambiae,AG1000G-BF-B,Burkina Faso,2014,Bana,gambiae,11.233,-4.472,0.025867,0.002333,15
12,AG1000G-BF-B.Pala.2014.gambiae,AG1000G-BF-B,Burkina Faso,2014,Pala,gambiae,11.150,-4.235,0.027188,0.002188,16
